# KR260 DMA Capture Buffer

## Setup

### Mounting Development Machine
This notebook utilizes `sshfs` to mount the development directory on the KR260 board to avoid transferring files bewteen machines.  

The development machine that was used to build the KR260 FPGA image is named `dev-wks`.  To mount the development directory on `dev-wks`, first will need to create a folder to mount the directory, then execute the following from the Jupyter Terminal:

```bash
cd /home/root/jupyter_notebooks
mkdir puch
sshfs sdr@dev-wks:/home/sdr/workspace/puch-workspace/HLS-QPSK-Demod-Baseband_002 /home/root/jupyter_notebooks/puch
```

If `sshfs` is not installed, then execute `sudo apt install sshfs` on the Jupyter Terminal.

### Unmounting Development Machine
To unmount the directory
` fusermount -u /home/root/jupyter_notebooks/dev-wks/`


### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install numpy==1.26.4
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

# Initilize

#### Bokeh Plot Function

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
import numpy as np
import math


def plot_time(in_signal,out_signal=None):
    t = np.linspace(0,len(in_signal),len(in_signal))
    output_notebook()
    p = figure(title = "Input & Output Signal")
    
    if out_signal is not None:
        p.line(t,out_signal,legend_label="Output Signal",line_color="red",line_width=1)
    p.line(t,in_signal,legend_label="Input Signal",line_color="blue",line_width=3)
    show(p)

#### Load Overlay

In [ ]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time

#import Python library from repo:
import sys
sys.path.append('/root/jupyter_notebooks/puch/')
import fpga.py.puch as puch
import fpga.lib.timestamp.sw.timestamp_regmap as timestamp_regmap
import fpga.lib.led_reg.sw.led_regmap as led_regmap
import fpga.lib.DMA_Data_Capture.sw.dma_data_capture_regmap as dma_capture_regmap

# Load the overlay
overlay = Overlay('../../overlays/KR260_DMA_Capture/output/kr260_dma_capture.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0
led_module   = overlay.led_reg_0
timestamp    = overlay.Timestamp_0
dma_capture  = overlay.DMA_Data_Capture_Top_0

print("Done Init System")

##### Display Time Stamp Register
The time stamp is burned into the FPGA during the build process

In [ ]:
print("FPGA Build Timestamp:  " + puch.get_timestamp_str(timestamp))

##### Toggle USER_LED[1:0] on/off

In [ ]:
for i in range(10):
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x1)
    time.sleep(0.25)
    led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x2)
    time.sleep(0.25)
led_module.mmio.write(led_regmap.RegMap.USER_LEDS_ADDR,0x0)

### Read DMA Data Capture Buffer

In [23]:
# Read the number of samples capture to use for DMA
Capture_Length       = 128

# Reset DMA Capture Buffer and configure a capture

dma_capture.mmio.write(dma_capture_regmap.RegMap.CAPTURE_LENGTH_ADDR, Capture_Length)
dma_capture.mmio.write(dma_capture_regmap.RegMap.FIFO_FLUSH_ADDR,1)

print("DMA Reset                     : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_FLUSH_ADDR)))
dma_capture.mmio.write(dma_capture_regmap.RegMap.FIFO_FLUSH_ADDR,0)
print("DMA Reset                     : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_FLUSH_ADDR)))

# Configure DMA Capture Buffer
print("DMA FIFO Depth                : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.MAX_DEPTH_ADDR)))
print("DMA FIFO Capture Length       : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.CAPTURE_LENGTH_ADDR)))
print("DMA FIFO Write Pointer        : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_WR_PTR_ADDR)))
print("DMA FIFO Read Pointer         : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_RD_PTR_ADDR)))

# Trigger DMA Capture 
print("Triggered DMA Capture...")
dma_capture.mmio.write(dma_capture_regmap.RegMap.CAPTURE_STB_ADDR,1)

print("DMA FIFO Depth                : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.MAX_DEPTH_ADDR)))
print("DMA FIFO Capture Length       : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.CAPTURE_LENGTH_ADDR)))
print("DMA FIFO Write Pointer        : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_WR_PTR_ADDR)))
print("DMA FIFO Read Pointer         : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_RD_PTR_ADDR)))

# Allocate DMA Buffer and Read
out_buffer = allocate(shape=(Capture_Length,), dtype=np.int32)
dma.recvchannel.transfer(out_buffer)
print("Done tranfer out_buffer")
dma.recvchannel.wait()
print("Done Wait")
samples = out_buffer.astype(np.uint32)
print("Symbols in Buffer             : " + str(len(samples)))
out_buffer.close()
del out_buffer

print("FIFO Write Pointer            : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_WR_PTR_ADDR)))
print("FIFO Read Pointer             : " + str(dma_capture.mmio.read(dma_capture_regmap.RegMap.FIFO_RD_PTR_ADDR)))

DMA Reset                     : 1
DMA Reset                     : 0
DMA FIFO Depth                : 32768
DMA FIFO Capture Length       : 128
DMA FIFO Write Pointer        : 0
DMA FIFO Read Pointer         : 0
Triggered DMA Capture...
DMA FIFO Depth                : 32768
DMA FIFO Capture Length       : 128
DMA FIFO Write Pointer        : 125
DMA FIFO Read Pointer         : 125
Done tranfer out_buffer
Done Wait
Symbols in Buffer             : 128
FIFO Write Pointer            : 1
FIFO Read Pointer             : 1


In [12]:
samples

PynqBuffer([32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18,
            17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,
             2,  1], dtype=uint32)

In [8]:
dma.register_map

RegisterMap {
  MM2S_DMACR = Register(RS=0, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=0, IRQDelay=0),
  MM2S_DMASR = Register(Halted=0, Idle=0, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SGDecErr=0, IOC_Irq=0, Dly_Irq=0, Err_Irq=0, IRQThresholdSts=0, IRQDelaySts=0),
  MM2S_CURDESC = Register(Current_Descriptor_Pointer=0),
  MM2S_CURDESC_MSB = Register(Current_Descriptor_Pointer=0),
  MM2S_TAILDESC = Register(Tail_Descriptor_Pointer=0),
  MM2S_TAILDESC_MSB = Register(Tail_Descriptor_Pointer=0),
  MM2S_SA = Register(Source_Address=0),
  MM2S_SA_MSB = Register(Source_Address=0),
  MM2S_LENGTH = Register(Length=0),
  SG_CTL = Register(SG_CACHE=0, SG_USER=0),
  S2MM_DMACR = Register(RS=1, Reset=0, Keyhole=0, Cyclic_BD_Enable=0, IOC_IrqEn=0, Dly_IrqEn=0, Err_IrqEn=0, IRQThreshold=1, IRQDelay=0),
  S2MM_DMASR = Register(Halted=0, Idle=1, SGIncld=0, DMAIntErr=0, DMASlvErr=0, DMADecErr=0, SGIntErr=0, SGSlvErr=0, SG

### Reset the DMA Controller
AXI DMA LogiCORE IP Product Guide: https://docs.amd.com/r/en-US/pg021_axi_dma/MM2S_DMACR-MM2S-DMA-Control-Register-Offset-00h

In [ ]:
# Initiate the DMA Reset for both RD/WR
dma.register_map.MM2S_DMACR.Reset = 1
dma.register_map.S2MM_DMACR.Reset = 1

# Re-enable the DMA Channel
dma.register_map.MM2S_DMACR.RS = 1
dma.register_map.S2MM_DMACR.RS = 1

# Complete it again
dma.recvchannel.stop()
dma.sendchannel.stop()
dma.recvchannel.start()
dma.sendchannel.start()

# clear buffers
out_buffer.close()
del out_buffer

In [ ]:
dma.recvchannel.idle
dma.sendchannel.running

In [ ]:
in_buffer.close()
out_buffer.close()